### Modelo de Flujo de Caja (Cash Flow)

Este notebook contiene dos modelos de analisis financiero que tienen como objeto de estudio los diferentes modelo de VE que se encuentran en el Dataset "ElectricCarData_Norm.csv". Estos modelos estudian el comportamiento financiero de cada prototipo del vehiculo materializado en el calculo de un Flujo de Caja (Cash Flow) proyectado a 5 años, e indicadores financiero como: ROI (Return on Investment), IR (Indice de Rentabilidad) y Payback Period.

El primer modelo recibe como parametro una instancia (modelo de VE) del dataset y devuelve como resultado el comportamiento de este en relacion a las variables financieras anteriormente expuestas y tambien el del un auto convencional para obtener un analisis comparativo y poder evaluar diferencias y el costo de oportunidad de estas dos variantes.


Importamos las librerias necesarias.

In [1]:
import pandas as pd 
import numpy_financial as npf

Importamos el dataset que contiene las instancias y variables necesarias. (Este dataset contiene los valores futuros, producto del modelo de ML)

In [2]:
future_data = pd.read_csv(r"..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Output_Anualized.csv")
future_data = future_data[future_data['industry']=='FHV - High Volume']

future_data.rename(columns={
    'anual_income_per_vehicle': 'Income_per_Vehicle (USD)',
    'anual_distance_per_vehicle': 'Miles_per_Vehicle',
    'anual_total_co2_emission': 'Total_CO2_Emissions'
}, inplace=True)
future_data =  future_data[future_data['year']>=2025]

# Dataset de VE
df_autos = pd.read_csv(r'..\datasets\2. Depurados\ElectricCarData_Clean.csv')#"df_modelo.csv")
df_autos.rename(columns={
    'brand': 'Brand',
    'model': 'Model',
    'efficiency_whkm': 'Efficiency (Wh/km)'
}, inplace=True)

df_autos['Efficiency (kWh/mile)'] = df_autos['Efficiency (Wh/km)']/1000* 0.1688666667 
df_autos['Precio Dolar'] = df_autos['price_euro'] * 1.06

Visualizamos el Dataset.

In [3]:
future_data.head(5)

,industry,year,Income_per_Vehicle (USD),Miles_per_Vehicle,Total_CO2_Emissions
4,FHV - High Volume,2025,65357.057024,11762.559488,390936.358803
5,FHV - High Volume,2026,64802.759539,11703.935721,391932.707891
6,FHV - High Volume,2027,64302.321066,11656.524791,392929.708332
7,FHV - High Volume,2028,63485.517739,11556.919908,394247.069542
8,FHV - High Volume,2029,63408.428262,11587.083884,395004.364135


In [4]:
future_data.rename(columns={
    'anual_income_per_vehicle': 'Income_per_Vehicle (USD)',
    'anual_distance_per_vehicle': 'Miles_per_Vehicle',
    'anual_total_co2_emissio': 'Total_CO2_Emissions'
}, inplace=True)
future_data =  future_data[future_data['year']>=2025]

Utilizamos como instrumento de ponderacion la media o promedio para tener un primer acercamiento y entendimiento a las variables (ingreso anual y millas recorridas al año). 

In [5]:
promedio_ingresos = round(future_data["Income_per_Vehicle (USD)"].mean(),2)
promedio_millas = round(future_data["Miles_per_Vehicle"].mean(),2)
print(f'El promedio de ingresos anuales es: {promedio_ingresos}')
print(f'El promedio de millas recorridas anualmente es : {promedio_millas}')

El promedio de ingresos anuales es: 59327.19
El promedio de millas recorridas anualmente es : 10873.13


Se asigna una variable a cada dato.

In [6]:
año = 2025
for _, row in future_data.iterrows():  # iterrows() en lugar de interrows()
    globals()[f'Ingresos_año_{año}'] = round(row["Income_per_Vehicle (USD)"],2)
    print(f'Los ingresos para el año {año} son {globals()[f"Ingresos_año_{año}"]}')
    año += 1

Los ingresos para el año 2025 son 65357.06
Los ingresos para el año 2026 son 64802.76
Los ingresos para el año 2027 son 64302.32
Los ingresos para el año 2028 son 63485.52
Los ingresos para el año 2029 son 63408.43
Los ingresos para el año 2030 son 63019.9
Los ingresos para el año 2031 son 62664.8
Los ingresos para el año 2032 son 61984.98
Los ingresos para el año 2033 son 62013.27
Los ingresos para el año 2034 son 61726.06
Los ingresos para el año 2035 son 19833.94


In [7]:
ingresos_años = []
for _, row in future_data.iterrows():
    ingresos_años.append(round(row["Income_per_Vehicle (USD)"],2))
print(ingresos_años)

[65357.06, 64802.76, 64302.32, 63485.52, 63408.43, 63019.9, 62664.8, 61984.98, 62013.27, 61726.06, 19833.94]


In [8]:
año = 2025
for _, row in future_data.iterrows():
    globals()[f'Millas_Año_{año}'] = round(row["Miles_per_Vehicle"],2)
    print(f'Las Millas recorridas en el año {año} son {globals()[f"Millas_Año_{año}"]}')
    año+=1

Las Millas recorridas en el año 2025 son 11762.56
Las Millas recorridas en el año 2026 son 11703.94
Las Millas recorridas en el año 2027 son 11656.52
Las Millas recorridas en el año 2028 son 11556.92
Las Millas recorridas en el año 2029 son 11587.08
Las Millas recorridas en el año 2030 son 11563.32
Las Millas recorridas en el año 2031 son 11546.49
Las Millas recorridas en el año 2032 son 11474.36
Las Millas recorridas en el año 2033 son 11527.96
Las Millas recorridas en el año 2034 son 11525.51
Las Millas recorridas en el año 2035 son 3699.81


In [9]:
millas_años = []
for _, row in future_data.iterrows():
    millas_años.append(round(row["Miles_per_Vehicle"],2))
print(millas_años)

[11762.56, 11703.94, 11656.52, 11556.92, 11587.08, 11563.32, 11546.49, 11474.36, 11527.96, 11525.51, 3699.81]


In [10]:
print(ingresos_años)
print(millas_años)

[65357.06, 64802.76, 64302.32, 63485.52, 63408.43, 63019.9, 62664.8, 61984.98, 62013.27, 61726.06, 19833.94]
[11762.56, 11703.94, 11656.52, 11556.92, 11587.08, 11563.32, 11546.49, 11474.36, 11527.96, 11525.51, 3699.81]


En esta sección se elabora el mecanismo para el cálculo del modelo de Flujo de Caja (Cash Flow) y los demas indicadores financieros.

A continuación se presentaran las siguientes características:

- Debido a que son dos tipos de autos con caracteristicas distintas se crean 2 clases para cada variante, que heredan de la clase madre «Auto».
- Las caracteristicas que estructuran la clase «Auto Convencional» corresponden a una ponderacion que representa un propotipo de auto convencional promedio
- Se crean funciones para el calculo de las variables financieras anteriormente mencionadas.
- El objeto de análisis es un modelo de VE que compone el dataset y tambien de un prototipo promedio de un auto convencional.
- El resultado es el dictámen financiero compuesto por las variables financieras expuestas.
- Obtención de análisis comparativo entre un VE y un auto convencional.
- Análisis Financiero de la totalidad de VE y ranking de los prototipos más rentables.

### Modelo de Flujo de Caja (Cash Flow) #1
Análisis Financiero personalizado (por modelo VE)

In [11]:
# Se crea la clase Auto y subclases para AutoConvencional y AutoElectrico
class Auto:
    def __init__(self,tasa_descuento, precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, 
                 ingresos_anuales):
        self.precio = precio
        self.iva = iva
        self.licencia_1 = licencia_1
        self.licencia_2 = licencia_2
        self.placa = placa
        self.inspeccion = inspeccion
        self.seguro = seguro
        self.recorrido_anual = recorrido_anual
        self.ingresos_anuales = ingresos_anuales
        self.tasa_descuento = tasa_descuento
    
    def inversion_inicial(self):
        raise NotImplementedError
    
    def costos_operativos(self):
        raise NotImplementedError
    
    def flujo_neto_anual(self):
        return self.ingresos_anuales - self.costos_operativos()
    
    def flujo_caja_proyectado(self, años=7):
        
        # Inversión inicial negativa porqué es el año 0
        inversion_inicial = -self.inversion_inicial()
        
        ingresos_año = [0]
        costos_año = [0]
        flujo_neto = [inversion_inicial]  # Año 0: 
        flujo_descuento = [inversion_inicial]  # Año 0 Inversión inicial negativa
        ingresos_descuento_año = [0]
        
        # Se calcula el flujo neto anual
        for año in range(1, años + 1):
            self.ingresos_anuales = ingresos_años[año-1]
            self.recorrido_anual = millas_años[año-1]
            flujo_neto_actual = self.flujo_neto_anual()
            flujo_neto.append(flujo_neto_actual)
            flujo_descuento_actual = flujo_neto_actual / (1 + self.tasa_descuento) ** año
            flujo_descuento.append(flujo_descuento_actual)
            ingresos_año.append(self.ingresos_anuales)
            ingresos_descuento_año.append(ingresos_años[año-1]  / (1 + self.tasa_descuento) ** año)
            costos_año.append(self.costos_operativos())
        
        # Se crea el DataFrame de resultados
        df_flujo_caja = pd.DataFrame({
            'Año': range(0, años + 1),
            'Ingresos': ingresos_año, 
            'Costos Operativos': costos_año,
            'Flujo Neto': flujo_neto,
            'Flujo Neto Descontado': flujo_descuento,
            'Ingresos Descontado': ingresos_descuento_año        
        })
        
        # Calculo el flujo total descontado para el ROI
        return df_flujo_caja

    def calcular_roi(self, flujo_total):
        roi = flujo_total/ self.inversion_inicial() *100
        roi_mensual_simple = roi/7/12 #- self.inversion_inicial()
        #roi_mensual_compuesto = ((1+ flujo_total/ self.inversion_inicial())**(1/(7*12))-1)*100 #- self.inversion_inicial()
        return round(roi, 2),round(roi_mensual_simple, 2)#,round(roi_mensual_compuesto, 2)

    def calcular_ir(self, ingreso_total):
        ir = ingreso_total / abs(self.inversion_inicial())
        return round(ir, 2)

    def calcular_payback_period(self):
        flujo_acumulado = 0
        payback = 0
        for año in range(1, 8):
            flujo_anual = self.flujo_neto_anual()
            flujo_acumulado += flujo_anual
            if flujo_acumulado >= abs(self.inversion_inicial()):
                payback = año
                break
        return payback if payback > 0 else "No recuperado en 7 años"
    

class AutoConvencional(Auto): #Licencia_1 = unico pago, licencia_2 = anual
    def __init__(self, tasa_descuento, precio=36200, iva=0.08875, licencia_1=550,licencia_2=100, placa = 300,
                 seguro=5000, inspeccion = 150,
                 eficiencia=25 , mantenimiento=0.15, recorrido_anual=promedio_millas, #eficiencia en MPG / mantenimiento = USD x Milla
                 ingresos_anuales=promedio_ingresos):
        super().__init__(tasa_descuento,precio, iva, licencia_1, licencia_2, placa, inspeccion,
                         seguro, recorrido_anual, ingresos_anuales) # Se usa el super constructor para evitar redundancia ya que es una clase hija
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
    
    def inversion_inicial(self):
        return self.precio * (1 + self.iva) + self.licencia_1
    
    def costos_operativos(self):
        precio_gasolina_galon = 3.06 #USD x galon
        costo_gasolina_anual = (self.recorrido_anual / self.eficiencia) * precio_gasolina_galon
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        costo_anual_salario = 2342 * 20  # Precio medio de la hora de un conductor de autos convencionales.
        return costo_gasolina_anual + costo_mantenimiento + self.seguro + self.licencia_2 + self.placa + self.inspeccion + costo_anual_salario



class AutoElectrico(Auto): #Descuento de 5% al 10% en licencia ; Excencion del impuesto de venta
    def __init__(self,tasa_descuento, precio, eficiencia, iva=0, licencia_1=495,licencia_2=100, placa = 300,
                 seguro=1500, inspeccion = 100, mantenimiento=0.03, wallbox=600, 
                 recambio_bateria=5000, #mantenimiento = USD x Milla
                 recorrido_anual=promedio_millas, ingresos_anuales=promedio_ingresos): 
        super().__init__(tasa_descuento, precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, 
                         ingresos_anuales)
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
        self.wallbox = wallbox
        self.recambio_bateria = recambio_bateria
        self.tasa_descuento = tasa_descuento
        self.licencia_1=licencia_1
    
    def inversion_inicial(self):
        return self.precio + self.licencia_1 + self.wallbox
    
    def costos_operativos(self):
        precio_electricidad_kWh = 0.13 #USD x kHw
        costo_electricidad_anual = (self.recorrido_anual * self.eficiencia) * precio_electricidad_kWh
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        costo_anual_bateria = self.recambio_bateria / 12.5
        costo_anual_salario = 2342 * 22 # Promedio de horas de menejo de un conductor por año * Precio promedio de la hora de un chofer de taxis en NY para VE.
        return costo_electricidad_anual + costo_mantenimiento + costo_anual_bateria + self.seguro + self.licencia_2 + self.placa + self.inspeccion + costo_anual_salario

# Función para seleccionar el modelo del auto, calcular flujos e indicadores
def comparar_flujo_caja_y_medir_indicadores(modelo, tasa_descuento):
    # Se Filtra en el dataset para obtener el precio del modelo
    filtro_modelo = df_autos[df_autos['Model'] == modelo]
    
    if filtro_modelo.empty:
        print(f"Modelo '{modelo}' no encontrado en el dataset.")
        return None
    
    precio = filtro_modelo['Precio Dolar'].values[0]
    eficiencia = filtro_modelo['Efficiency (kWh/mile)'].values[0]
    
    # Instancio el auto eléctrico y el auto convencional
    auto_electrico = AutoElectrico(tasa_descuento, precio, eficiencia)
    auto_convencional = AutoConvencional(tasa_descuento)
    
    # Se calcula el flujo de caja proyectado
    flujo_electrico = auto_electrico.flujo_caja_proyectado()
    flujo_convencional = auto_convencional.flujo_caja_proyectado()
    
    # Sumo flujo neto descontado para cada tipo de auto (obtener total)
    flujo_descuento_total_electrico = round(flujo_electrico['Flujo Neto Descontado'].sum(),2)
    flujo_descuento_total_convencional = round(flujo_convencional['Flujo Neto Descontado'].sum(),2)

    # Sumo ingresos descontado para cada tipo de auto (obtener total)
    ingresos_descuento_total_electrico = round(flujo_electrico['Ingresos Descontado'].sum(),2)
    ingresos_descuento_total_convencional = round(flujo_convencional['Ingresos Descontado'].sum(),2)

    # Calculo del ROI, IR y Payback Period para cada auto
    roi_electrico, roi_mensual_electrico = auto_electrico.calcular_roi(flujo_descuento_total_electrico)
    roi_convencional = auto_convencional.calcular_roi(flujo_descuento_total_convencional)
    ir_electrico = auto_electrico.calcular_ir(ingresos_descuento_total_electrico)
    ir_convencional = auto_convencional.calcular_ir(ingresos_descuento_total_convencional)
    payback_electrico = auto_electrico.calcular_payback_period()
    payback_convencional = auto_convencional.calcular_payback_period()
    
    df_flujo_electrico = pd.DataFrame(flujo_electrico)
    df_flujo_convencional = pd.DataFrame(flujo_convencional)

    # Resultados
    print(f"Flujo de Caja Proyectado para {modelo} (Eléctrico):")
    print(df_flujo_electrico)
    
    print("\nFlujo de Caja Proyectado para Auto Convencional:")
    print(df_flujo_convencional)
    
    print("\nComparación Financiera:")
    print(f"Auto Eléctrico - Flujo Neto Descontado Total VNA: {flujo_descuento_total_electrico}, ROI: {roi_electrico}%, ROI_Mensual: {roi_mensual_electrico}%, IR: {ir_electrico} USD, Payback Period: {payback_electrico} años")
    print(f"Auto Convencional - Flujo Neto Descontado Total VNA: {flujo_descuento_total_convencional}, ROI: {roi_convencional}%, IR: {ir_convencional} USD, Payback Period: {payback_convencional} años")

Ejemplo de uso:

In [12]:
modelo_auto = 'Mii Electric '  # El modelo elegido
tasa_descuento = 0.10
comparar_flujo_caja_y_medir_indicadores(modelo_auto, tasa_descuento)

Flujo de Caja Proyectado para Mii Electric  (Eléctrico):
   Año  Ingresos  Costos Operativos    Flujo Neto  Flujo Neto Descontado  \
0    0      0.00           0.000000 -22431.740000          -22431.740000   
1    1  65357.06       54319.741247  11037.318753           10033.926139   
2    2  64802.76       54317.769027  10484.990973            8665.281796   
3    3  64302.32       54316.173622   9986.146378            7502.739578   
4    4  63485.52       54312.822665   9172.697335            6265.075702   
5    5  63408.43       54313.837373   9094.592627            5647.026487   
6    6  63019.90       54313.037988   8706.862012            4914.796618   
7    7  62664.80       54312.471757   8352.328243            4286.065044   

   Ingresos Descontado  
0             0.000000  
1         59415.509091  
2         53556.000000  
3         48311.284748  
4         43361.464381  
5         39371.646249  
6         35573.090625  
7         32156.950847  

Flujo de Caja Proyectado para Au

A continuacion se exponen los modelos contenidos en el dataset para agregarlos en le input de la funcion.

In [13]:
#df_autos["Model"].unique()

### Modelo de Flujo de Caja (Cash Flow) #2:
Ranking de rentabilidad financiera de todos los prototipos de VE contenidos en el dataset

In [14]:
# Definición de la función para calcular y comparar métricas
def calcular_metricas_para_todos_los_modelos(tasa):
    resultados = []
    resultados_conv = []

    # Se itera sobre cada modelo en el dataset
    for _, row in df_autos.iterrows(): #uso _, ya que no necesito el indice
        marca = row['Brand']
        modelo = row['Model']
        tipo_de_auto = row['body_style']
        segmento = row['segment']
        precio = row['Precio Dolar']
        eficiencia = row['Efficiency (kWh/mile)']
       
        # Se instancia AutoEléctrico y AutoConvencional con el precio del modelo actual
        auto_electrico = AutoElectrico(tasa_descuento, precio, eficiencia)
        auto_convencional = AutoConvencional(tasa_descuento)

        # Se calcula los flujos de caja proyectados para ambos tipos de autos
        inversión_electrico = auto_electrico.inversion_inicial()
        inversión_convencional = auto_convencional.inversion_inicial()
        
        # Se calcula los flujos de caja proyectados para ambos tipos de autos
        flujo_electrico = auto_electrico.flujo_caja_proyectado()
        flujo_convencional = auto_convencional.flujo_caja_proyectado()

        # Luego, se calcula el flujo neto descontado total
        flujo_descuento_total_electrico = flujo_electrico['Flujo Neto Descontado'].sum()
        flujo_descuento_total_convencional = flujo_convencional['Flujo Neto Descontado'].sum()

        # Sumo ingresos descontado para cada tipo de auto (obtener total)
        ingresos_descuento_total_electrico = round(flujo_electrico['Ingresos Descontado'].sum(),2)
        ingresos_descuento_total_convencional = round(flujo_convencional['Ingresos Descontado'].sum(),2)


        # Calculo ROI, IR y Payback Period
        roi_electrico,roi_mensual_electrico = auto_electrico.calcular_roi(flujo_descuento_total_electrico)
        ir_electrico = auto_electrico.calcular_ir(ingresos_descuento_total_electrico)
        payback_electrico = auto_electrico.calcular_payback_period()
        tir_electrico = round(npf.irr(flujo_electrico['Flujo Neto'])*100,2)

        # Calculo ROI, IR y Payback Period
        roi_convencional,roi_mensual_convencional  = auto_convencional.calcular_roi(flujo_descuento_total_convencional)
        ir_convencional = auto_convencional.calcular_ir(ingresos_descuento_total_convencional)
        payback_convencional = auto_convencional.calcular_payback_period()
        tir_convencional = round(npf.irr(flujo_convencional['Flujo Neto'])*100,2)


        # Se agg los resultados a la lista
        resultados.append({
            'Brand': marca,
            'Model': modelo,
            'Tipo de Auto': tipo_de_auto,
            'Segmento':segmento,
            'Efficiency (kWh/mile)' : eficiencia,
            'Precio (USD)': round(precio,1),
            'Inversión Inicial Total (USD)': inversión_electrico,
            'VNA (USD)': round(flujo_descuento_total_electrico,1),
            'TIR (%)':tir_electrico,
            'ROI (%)': roi_electrico,
            'ROI Mensualizado(%)': roi_mensual_electrico,
            'IR (USD)': ir_electrico,
            'Payback Period (Años)': payback_electrico
        })

        # Se agg los resultados a la lista
        resultados_conv.append({
            'Modelo': "Convencional",
            'Precio (USD)': 36200,
            'Inversión Inicial Total (USD)': inversión_convencional,
            'TIR (%)':tir_convencional,
            'ROI (%)': roi_convencional,
            'ROI Mensualizado(%)': roi_mensual_convencional,
            'IR (USD)': ir_convencional,
            'Payback Period (Años)': payback_convencional
        })

        # Se convierte a DataFrame
    df_resultados = pd.DataFrame(resultados)
    df_resultados_conv = pd.DataFrame(resultados_conv)
    # Se obtienen los 5 mejores con nlargest
    # top_5_modelos = df_resultados.nlargest(5, ['ROI (%)', 'IR (USD)', 'Flujo Neto Descontado (USD)'])
    df_resultados = df_resultados.sort_values(by="ROI (%)", ascending=False)
    df_resultados_conv = df_resultados_conv.sort_values(by="ROI (%)", ascending=False)

    return df_resultados, df_resultados_conv
    #return top_5_modelos

# Se invoca la función para calcular métricas y obtener los top 5 modelos
tasa_descuento = 0.10
df_resultados,df_resultados_conv = calcular_metricas_para_todos_los_modelos(tasa_descuento)

df_resultados.head(5)

,Brand,Model,Tipo de Auto,Segmento,Efficiency (kWh/mile),Precio (USD),Inversión Inicial Total (USD),VNA (USD),TIR (%),ROI (%),ROI Mensualizado(%),IR (USD),Payback Period (Años)
44,SEAT,Mii Electric,Hatchback,A,0.028032,21336.7,22431.74,24883.2,41.11,110.93,1.32,13.90,3
82,Smart,EQ fortwo coupe,Hatchback,A,0.028201,22670.2,23765.22,23548.4,38.05,99.09,1.18,13.12,3
17,Volkswagen,e-Up!,Hatchback,A,0.028032,22706.3,23801.26,23513.7,37.97,98.79,1.18,13.10,3
77,Smart,EQ forfour,Hatchback,A,0.029721,23351.8,24446.80,22855.7,36.58,93.49,1.11,12.75,3
43,Skoda,CITIGOe iV,Hatchback,A,0.028032,26006.0,27101.04,20213.9,31.57,74.59,0.89,11.50,4


In [15]:
df_resultados_conv.head(5)

,Modelo,Precio (USD),Inversión Inicial Total (USD),TIR (%),ROI (%),ROI Mensualizado(%),IR (USD),Payback Period (Años)
0,Convencional,36200,39962.75,11.03,3.23,0.04,7.8,6
64,Convencional,36200,39962.75,11.03,3.23,0.04,7.8,6
74,Convencional,36200,39962.75,11.03,3.23,0.04,7.8,6
73,Convencional,36200,39962.75,11.03,3.23,0.04,7.8,6
72,Convencional,36200,39962.75,11.03,3.23,0.04,7.8,6


In [16]:
df_resultados.to_csv('Analisis_Financiero_VE.csv', index=False) 